In [4]:
import sys
import gc
import os
#base_dir = "drive/My Drive/jp_en_translation/".replace("/",os.sep)
base_dir = "../"
emb_dir = "drive/My Drive/util_data/".replace("/",os.sep)
sys.path.append("/Users/ueki/Desktop/work/jp_en_translation")
sys.path.append("/Users/ueki/Desktop/JP_EN_Transalation")
sys.path.append(base_dir)
sys.path.append("drive/My Drive/jp_en_translation".replace("/",os.sep))

In [5]:
from models.Transformer import Model
from utils.LangEn import LangEn
from utils.LangJa import LangJa
from utils.build_emb import build_en_emb,build_jp_emb
from utils.preprocess import loadLangs
import numpy as np
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
import torch

Using TensorFlow backend.


In [8]:
config={
    "corpus_file":base_dir+"data/test.txt",
    "en_col":"description_en",
    "jp_col":"description_jp",
    "mask_token":0,
    "SOS_token":1,
    "EOS_token":2,
    "UNK_token":3,
    "max_features":8000,
    "MAX_LENGTH":20,
    "train_size":15000,
    "val_size":300,
    "batch_size":128,
    "epochs":1,
    "maxlen_enc":20,
    "maxlen_dec":20,
    "n_hidden":500,
    "input_dim":5000,
    "jp_voc":8000,
    "en_voc":5000,
    "output_dim":8000,
    "emb_dim":300,
    "validation_split":0.01,

    "translate_length":20,
    "en_W2V_FILE" : emb_dir+"GoogleNews-vectors-negative300.bin.gz",
    "jp_W2V_FILE":emb_dir+"ja_data/ja.bin",
    "src":"en",
    "trg":"jp",
    "learning_rate":0.001,
    "n_heads":5,
    "n_layer":4,
    "param_dir":base_dir+"param/test_sub.pth"
}

# train

In [9]:
data=loadLangs(config)

reading lines


In [23]:
len(data)

15000

In [10]:
val_data = data[config["train_size"]:config["train_size"]+config["val_size"]]
data = data[:config["train_size"]]

In [11]:
EN_lang = LangEn(config)
JP_lang = LangJa(config)

stdbuf was not found; communication with perl may hang due to stdio buffering.


In [12]:
for s in data[config["en_col"]]:
    EN_lang.addSentence(s)

In [13]:
for s in data[config["jp_col"]]:
    JP_lang.addSentence(s)

## input の加工

In [14]:
if config["src"]=="jp":
    src_col=config["jp_col"]
    trg_col=config["en_col"]
    Langs={"src":JP_lang,"trg":EN_lang}
else:
    src_col=config["en_col"]
    trg_col=config["jp_col"]
    Langs={"trg":JP_lang,"src":EN_lang}

In [15]:
input_en = data[src_col]

In [16]:
input_source_lang=data[src_col].apply(lambda x:Langs["src"].word2id(x))
input_target_lang=data[trg_col].apply(lambda x:Langs["trg"].word2id(x,target=True))

In [17]:
val_source = val_data[src_col].apply(lambda x:Langs["src"].word2id(x))
val_target = val_data[trg_col].apply(lambda x:Langs["trg"].word2id(x))

In [18]:
input_source_padded=sequence.pad_sequences(input_source_lang,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")
input_target_padded=sequence.pad_sequences(input_target_lang,maxlen=config["MAX_LENGTH"]+1,padding="post",truncating="post")

In [19]:
val_source_padded=sequence.pad_sequences(val_source,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")
val_target_padded=sequence.pad_sequences(val_target,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [22]:
len(input_source_lang)

15000

## create embedding

In [16]:
#en_emb = build_en_emb(config,EN_lang.word2index)

In [17]:
#ja_emb = build_jp_emb(config,JP_lang.word2index)

In [18]:
#if config["src"]=="en":
 #   emb={"src_emb":en_emb,"trg_emb":ja_emb}
#else:
 #   emb={"src_emb":ja_emb,"trg_emb":en_emb}

In [19]:
#print("enc->",emb["trg_emb"].shape)
#print("dec->",emb["src_emb"].shape)

In [21]:
#trainer = Seq2Seq(config,enc_emb=emb["src_emb"],dec_emb=emb["trg_emb"])
trainer = Model(config)

In [21]:
#trainer.trainIters(input_source_padded,input_target_padded,val_source_padded,val_target_padded)

# test

In [22]:
val_src=val_source_padded[:1]

In [23]:
trainer.translate(val_src)

/Users/ueki/Desktop/work/jp_en_translation/models/Transformer.py:386: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs[:i], dtype=torch.long).view(1, -1).to(device)


[1,
 5108,
 2849,
 1934,
 2552,
 3536,
 1180,
 1384,
 3337,
 842,
 5934,
 5251,
 3465,
 3683,
 2767,
 3222,
 3890,
 5195,
 5330]

In [24]:
ret = trainer.translate(val_src)
Langs["trg"].id2word(ret)

['SOS',
 '部分',
 'のぞみ',
 '病気',
 '応じ',
 '北部',
 '作れる',
 '1958',
 '着用',
 '逃げ出し',
 '様子',
 '後片付け',
 'ポテチ',
 '何でも',
 '丘',
 'グチャグチャ',
 'コンサート',
 '相づち',
 '当たり前']

In [25]:
ret=trainer.translate_beam(val_src,3)

/Users/ueki/Desktop/work/jp_en_translation/models/Transformer.py:414: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cand[:i], dtype=torch.long).view(1, -1).to(device)


In [26]:
for r in ret:
    print(Langs["trg"].id2word(r))

['SOS', '示す', '金', '両替', '完璧', '出かけ', '裏庭', '後部', 'ひっかかり', '記憶', '申し込み', '世間', 'こらえる', '市内', '淡い', '戦慄', '許さ', '1840', '出来る', '雪']
['SOS', '示す', '金', '両替', '完璧', '出かけ', '裏庭', '後部', 'ひっかかり', '記憶', '申し込み', '世間', 'こらえる', '市内', '淡い', '戦慄', '許さ', '1840', '出来る', '雪']
['SOS', '示す', '金', '両替', '完璧', '出かけ', '裏庭', '後部', 'ひっかかり', '記憶', '申し込み', '世間', 'こらえる', '市内', '淡い', '戦慄', '許さ', '1840', '出来る', '雪']


In [27]:
trainer.save_model()

In [28]:
trainer.load_model()